# Ideal Processing Pipeline for Consistent CTCF in Fluorescence Microscopy

Here's a comprehensive pipeline for analyzing fluorescence microscopy images with consistent CTCF measurement across different conditions and microscopes:



In [9]:
## Import necessary libraries
import time, os, sys
import traceback
from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg
import tifffile as tiff
from PIL import Image

import numpy as np
import pandas as pd
from skimage import measure, draw, exposure
from skimage.transform import resize
from skimage.segmentation import find_boundaries

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import scipy.stats

from tqdm import tqdm
import torch
import glob
import gc

mpl.rcParams['figure.dpi'] = 200

from AutoImgUtils import * 

In [2]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics, denoise
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:42:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0
Wed Apr  9 20:44:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.61                 Driver Version: 572.61         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN RTX             WDDM  |   00000000:81:00.0 Off |                  N/A |
| 41% 

## Function Definitions

### Visualization functions
Here the visualization functions are defined, that can be used to save plots regarding the background substraction as well as the segmentation results for quality control

In [14]:
def visualize_background_mask(channel_image, bg_model, output_path, n_components=3, enhance_contrast=True):
    """Visualize background mask from GMM model with distribution plots"""
    # Create figure with 4 subplots (2x2 grid)
    fig, axes = plt.subplots(2, 2, figsize=(15, 12), gridspec_kw={'height_ratios': [3, 1]})
    
    # Enhance contrast for visualization if requested
    if enhance_contrast:
        # Use percentile-based contrast stretching (robust to outliers)
        p_low, p_high = 2, 98  # Percentiles for contrast stretching
        display_img = exposure.rescale_intensity(
            channel_image, 
            in_range=tuple(np.percentile(channel_image, (p_low, p_high))),
            out_range='dtype'
        )
    else:
        display_img = channel_image
    
    # Original image with enhanced contrast
    axes[0, 0].imshow(display_img, cmap='gray')
    axes[0, 0].set_title('Original Channel' + (' (Contrast Enhanced)' if enhance_contrast else ''))
    plt.colorbar(axes[0, 0].get_images()[0], ax=axes[0, 0])
    
    # Background mask
    axes[0, 1].imshow(bg_model['mask'], cmap='hot')
    axes[0, 1].set_title(f'Background Mask\nMean: {bg_model["mean"]:.2f}, Std: {bg_model["std"]:.2f}')
    plt.colorbar(axes[0, 1].get_images()[0], ax=axes[0, 1])
    
    # Original with background highlighted
    norm_img = (display_img - np.min(display_img)) / (np.max(display_img) - np.min(display_img))
    rgb_img = np.stack([norm_img, norm_img, norm_img], axis=-1)
    
    # Highlight background in red
    rgb_img[:,:,0][bg_model['mask']] = 1.0  # Set red high for background
    rgb_img[:,:,1][bg_model['mask']] = 0.0  # Set green low for background
    rgb_img[:,:,2][bg_model['mask']] = 0.0  # Set blue low for background
    
    axes[1, 0].imshow(rgb_img)
    axes[1, 0].set_title('Background Regions (Red)')
    
    # Plot intensity histogram with GMM distributions
    if 'gmm' in bg_model:
        gmm = bg_model['gmm']
        flat_img = channel_image.flatten()
        
        # Plot histogram
        hist_range = (np.min(flat_img), np.max(flat_img))
        n_bins = 100
        axes[1, 1].hist(flat_img, bins=n_bins, range=hist_range, density=True, 
                       alpha=0.6, color='gray', label='Pixel Intensity')
        
        # Create x values for plotting GMM curves
        x = np.linspace(hist_range[0], hist_range[1], 1000)
        x_reshaped = x.reshape(-1, 1)
        
        # Plot the individual components
        colors = ['blue', 'green', 'red', 'purple', 'orange']
        bg_component = np.argmin(gmm.means_.flatten())
        
        for i in range(gmm.n_components):
            # Calculate component density
            weight = gmm.weights_[i]
            mean = gmm.means_[i, 0]
            std = np.sqrt(gmm.covariances_[i, 0, 0])
            
            # Create a normal distribution for this component
            y = weight * scipy.stats.norm.pdf(x, mean, std)
            
            # Plot with higher alpha for background component
            alpha = 0.8 if i == bg_component else 0.5
            label = f"Background (μ={mean:.1f})" if i == bg_component else f"Component {i+1} (μ={mean:.1f})"
            axes[1, 1].plot(x, y, color=colors[i % len(colors)], 
                          alpha=alpha, linewidth=2, label=label)
        
        axes[1, 1].set_title('Pixel Intensity Distribution')
        axes[1, 1].set_xlabel('Pixel Value')
        axes[1, 1].set_xscale('log')
        axes[1, 1].set_ylabel('Density')
        axes[1, 1].legend()
        
    plt.tight_layout()
    plt.savefig(output_path, dpi=200)
    plt.close('all')

def create_visualization(image, masks, measurements, output_path, debug=False):
    """
    Create multi-panel visualization for QC with optimized memory usage and enhanced contrast
    
    Parameters:
    - image: Multi-channel image
    - masks: Cell segmentation masks
    - measurements: Cell measurements
    - output_path: Where to save the visualization
    - debug: Enable detailed timing and progress tracking
    """
    try:
        if debug:
            start_time = time.time()
            print(f"Starting visualization for {output_path}...")
        
        # Force garbage collection before starting
        gc.collect()
        
        # Get image dimensions and channel count
        h, w, n_channels = image.shape
        
        # Calculate reasonable figure size to avoid excessive memory usage
        max_dim = 2000  # Maximum dimension in pixels
        scale_factor = min(1.0, max_dim / max(h, w))
        
        # Create figure without displaying (reduces memory usage)
        dpi = 300  # Mantain good quality
        fig_width = (n_channels + 1) * 4  # 4 inches per panel
        fig_height = 4  # Fixed height
        
        # Use Figure directly instead of pyplot to avoid memory leaks
        fig = Figure(figsize=(fig_width, fig_height), dpi=dpi)
        canvas = FigureCanvasAgg(fig)
        
        if debug:
            print(f"Created figure with size {fig_width}x{fig_height} inches at {dpi} DPI")
            print(f"Processing {n_channels} channels and {len(measurements)} cells")
        
        # Create subplot grid
        grid = fig.add_gridspec(1, n_channels + 1)
        
        # Plot segmentation mask (first panel) with enhanced contrast
        if debug:
            print("Rendering segmentation mask...")
        
        ax = fig.add_subplot(grid[0, 0])
        
        # Convert mask to float for better visualization
        mask_display = (masks > 0).astype(float)
        # Apply contrast enhancement to make it more visible
        mask_display = exposure.equalize_adapthist(mask_display)
        ax.imshow(mask_display, cmap='viridis')  # Use viridis for better contrast
        ax.set_title('Cell Segmentation (Enhanced)')
        ax.axis('off')  # Turn off axes to save memory
        
        # Only add labels for a subset of cells
        if len(measurements) > 0:
            if debug:
                print("Adding cell labels...")
            # Select a random subset of 50 cells or fewer if there are less than 50
            num_labels = min(50, len(measurements))
            # Use numpy's random choice if measurements is a list, otherwise select first num_labels
            if isinstance(measurements, list):
                indices = np.random.choice(len(measurements), num_labels, replace=False)
                label_subset = [measurements[i] for i in indices]
            else:
                label_subset = measurements[:num_labels]
                
            for cell in label_subset:
                y, x = cell['centroid']
                ax.text(x, y, str(cell['label']), color='red', fontsize=5)
        
        # Process channels with progress tracking
        channel_range = range(n_channels)
        if debug:
            from tqdm import tqdm
            channel_range = tqdm(channel_range, desc="Processing channels")
        
        for ch_idx, ch in enumerate(channel_range):
            if debug:
                ch_start = time.time()
                
            # Create subplot for this channel
            ax = fig.add_subplot(grid[0, ch_idx + 1])
            
            # Get channel data and apply adaptive contrast enhancement
            channel_data = image[:,:,ch].copy()  # Make a copy to avoid modifying original
            
            # Adaptive histogram equalization - best for visualizing local features
            enhanced_data = exposure.equalize_adapthist(channel_data, clip_limit=0.03)
            
            # Display the image
            ax.imshow(enhanced_data, cmap='hot')
            ax.set_title(f'Channel {ch+1} (Enhanced)')
            ax.axis('off')  # Turn off axes to save memory
            
            # Show cell boundaries efficiently
            boundaries = find_boundaries(masks > 0)
            ax.imshow(boundaries, alpha=0.3, cmap='cool')
            
            # Free memory
            del channel_data
            del enhanced_data
            del boundaries
            
            if debug:
                print(f"  Channel {ch+1} rendered in {time.time() - ch_start:.2f}s")
        
        # Adjust layout and save
        if debug:
            print("Saving figure...")
            save_start = time.time()
            
        fig.tight_layout()
        fig.savefig(output_path, bbox_inches='tight')
        
        # Clean up matplotlib resources explicitly
        fig.clf()
        canvas.renderer.clear()
        del fig
        del canvas
        
        # Force garbage collection
        gc.collect()
        
        if debug:
            print(f"Visualization saved in {time.time() - save_start:.2f}s")
            print(f"Total visualization time: {time.time() - start_time:.2f}s")
            
    except Exception as e:
        print(f"Error in visualization: {str(e)}")
        traceback.print_exc()
        
        # Ensure cleanup even on error
        if 'fig' in locals():
            fig.clf()
            del fig
        if 'canvas' in locals():
            canvas.renderer.clear()
            del canvas
        gc.collect()

### Background substraction and CellPose based Segmentation Functions
Here the main functions for background substraction and cell segmentation based on de cyto3 model are defined

In [4]:
def process_experiments_batch(main_directory, config):
    """
    Process all experiments in the main directory with improved memory management
    """
    # Create results directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = os.path.join(main_directory, f"CTCF_Analysis_{timestamp}")
    os.makedirs(results_dir, exist_ok=True)
    
    # Find all experiment folders
    experiment_folders = [f.path for f in os.scandir(main_directory) if f.is_dir() 
                         and not f.name.startswith('.') and not "CTCF_Analysis" in f.name]
    
    # Process each experiment folder
    for experiment_folder in experiment_folders:
        exp_name = os.path.basename(experiment_folder)
        print(f"\nProcessing experiment: {exp_name}")
        
        # Create experiment results folder
        exp_results_dir = os.path.join(results_dir, exp_name)
        os.makedirs(exp_results_dir, exist_ok=True)
        
        # Process all images in experiment folder
        image_files = [f for f in os.listdir(experiment_folder) 
                      if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
        
        # Create a DataFrame for experiment results instead of list to save memory
        exp_results_df = pd.DataFrame()
        
        for image_file in image_files:
            # Full image processing pipeline for each image
            img_result = process_single_image(
                os.path.join(experiment_folder, image_file),
                exp_results_dir,
                config
            )
            
            # Append result to DataFrame directly
            exp_results_df = pd.concat([exp_results_df, pd.DataFrame([img_result])], ignore_index=True)
            
            # Force garbage collection after each image
            gc.collect()
        
        # Save experiment results
        exp_results_df.to_csv(os.path.join(exp_results_dir, f"{exp_name}_results.csv"), index=False)
        
        # Write to batch summary file incrementally instead of keeping in memory
        batch_summary_path = os.path.join(results_dir, "batch_summary.csv")
        if os.path.exists(batch_summary_path):
            exp_results_df.to_csv(batch_summary_path, mode='a', header=False, index=False)
        else:
            exp_results_df.to_csv(batch_summary_path, index=False)
        
        # Clear DataFrame to free memory
        del exp_results_df
        gc.collect()
    
    print(f"All experiments processed. Results saved to: {results_dir}")
    return results_dir

def resample_image(image, scale_factor=0.5):
    """Resample image by the given scale factor"""
    
    # Get original dimensions
    h, w, c = image.shape
    
    # Calculate new dimensions
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    
    # Resize image
    resized = resize(image, (new_h, new_w, c), preserve_range=True, anti_aliasing=True)
    
    return resized.astype(image.dtype)


def process_single_image(image_path, output_dir, config):
    """Process a single fluorescence microscopy image with improved memory management"""
    try:
        # Load image and normalize channels
        image = tiff.imread(image_path)
        img_name = os.path.basename(image_path)
        img_base = os.path.splitext(img_name)[0]

        print(f"\nProcessing image: {img_name}")

        # Extract debug flag from config
        debug = config.get('debug', False)
        
        # Move the shortest axis (channels) to the last index
        shortest_axis = np.argmin(image.shape)
        image = np.moveaxis(image, shortest_axis, -1)
        
        # Extract configuration
        channels_of_interest = config.get('channels_of_interest', list(range(image.shape[-1])))
        
        # 1. BACKGROUND ESTIMATION USING GMM
        print("Estimating background using GMM...")
        bg_models = {}
        for ch in tqdm(range(image.shape[-1]), desc="Background estimation", leave=False):
            # Process one channel at a time to reduce memory usage
            channel_data = image[:,:,ch].copy()  # Make a copy to avoid reference issues
            bg_models[ch] = fast_estimate_background_gmm(channel_data)
            
            # Save background mask visualization if needed
            if config.get('visualize_steps', True):
                visualize_background_mask(channel_data, bg_models[ch], 
                                         os.path.join(output_dir, f"{img_base}_bg_mask_ch{ch+1}.png"))
                plt.close('all')  # Ensure all plots are closed
            
            # Remove channel data from memory
            del channel_data
        
        print("Background estimation complete.")
        
        # 2. CELL SEGMENTATION USING CELLPOSE
        cell_masks = segment_cells_with_downsampling(image, config, bg_models)
        
        # 3. MEASURE CTCF FOR EACH CELL AND CHANNEL
        cell_measurements = measure_cells_ctcf(image, cell_masks, bg_models)
        
        # Store key results in a DataFrame and save immediately
        cell_df = pd.DataFrame([
            {
                'image': img_name,
                'cell_id': i,
                'area': cell['area'],
                **{f'channel_{ch+1}_ctcf': cell['ctcf'][ch] for ch in channels_of_interest},
                **{f'channel_{ch+1}_mean': cell['mean'][ch] for ch in channels_of_interest},
                'centroid_x': cell['centroid'][0],
                'centroid_y': cell['centroid'][1]
            }
            for i, cell in enumerate(cell_measurements)
        ])
        cell_df.to_csv(os.path.join(output_dir, f"{img_base}_cells.csv"), index=False)
        
        # 4. GENERATE VISUALIZATIONS - do this after saving measurements to free memory
        if debug:
            print(f"Starting visualization with debug mode...")
        
        create_visualization(image, cell_masks, cell_measurements, 
                             os.path.join(output_dir, f"{img_name}_analysis.png"),
                             debug=debug)
        
        # Explicitly close all plots and collect garbage
        plt.close('all')
        gc.collect()
        
        # 5. CLEANUP AND RETURN RESULTS
        # Create a minimal results dictionary with just the summary stats
        results = {
            'image_name': img_name,
            'total_cells': len(cell_measurements),
            'image_path': image_path,
        }
        
        # Add summarized measurements
        for ch in channels_of_interest:
            ch_ctcf = [cell['ctcf'][ch] for cell in cell_measurements]
            results[f'channel_{ch+1}_mean_ctcf'] = np.mean(ch_ctcf)
            results[f'channel_{ch+1}_median_ctcf'] = np.median(ch_ctcf)
            results[f'channel_{ch+1}_std_ctcf'] = np.std(ch_ctcf)
            
        # Clean up large objects
        del image, cell_masks, cell_measurements, bg_models, cell_df
        gc.collect()
        
        return results
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        traceback.print_exc()
        gc.collect()
        return {'image_name': os.path.basename(image_path), 'error': str(e), 'total_cells': 0}

def fast_estimate_background_gmm_gpu(channel_image, n_components=3, sample_ratio=0.1):
    """GPU-accelerated background estimation using GMM"""
    
    # Convert to torch tensor and flatten
    device = torch.device('cuda')
    flat_img_gpu = torch.from_numpy(channel_image.flatten()).float().to(device)
    
    # Random sampling on GPU for efficiency
    n_samples = max(10000, int(sample_ratio * flat_img_gpu.size(0)))
    indices = torch.randperm(flat_img_gpu.size(0), device=device)[:n_samples]
    sample_data_gpu = flat_img_gpu[indices].reshape(-1, 1).cpu().numpy()
    
    # The GMM calculation itself stays on CPU as sklearn doesn't support GPU
    # But data preparation and sampling are accelerated
    gmm = GaussianMixture(n_components=n_components, random_state=42, max_iter=100)
    gmm.fit(sample_data_gpu)
    
    # Process results on GPU where possible
    means = torch.tensor(gmm.means_.flatten(), device=device)
    bg_component = torch.argmin(means).item()
    bg_mean = means[bg_component].item()
    bg_std = float(np.sqrt(gmm.covariances_[bg_component].flatten()[0]))
    
    # Predict on GPU for mask creation
    # This is a hybrid approach: get scores from CPU model but process on GPU
    pixel_labels_cpu = gmm.predict(flat_img_gpu.cpu().reshape(-1, 1).numpy())
    pixel_labels_gpu = torch.from_numpy(pixel_labels_cpu).to(device)
    bg_mask = (pixel_labels_gpu == bg_component).reshape(channel_image.shape)
    
    # Return result including CPU-based GMM for compatibility
    return {
        'mean': bg_mean,
        'std': bg_std,
        'mask': bg_mask.cpu().numpy(),
        'gmm': gmm,
        'bg_percentage': (torch.sum(bg_mask).item() / bg_mask.numel() * 100),
        'component_means': gmm.means_.flatten(),
        'n_components': n_components,
        'bg_component': bg_component
    }

def fast_estimate_background_gmm(channel_image, n_components=3, sample_ratio=0.1, max_iter=100):
    """
    Faster background estimation using GMM with downsampling
    
    Parameters:
    - channel_image: 2D array with image channel
    - n_components: Number of GMM components
    - sample_ratio: Fraction of pixels to sample (smaller = faster)
    - max_iter: Maximum EM iterations
    """
    try:
        # Flatten image
        flat_img = channel_image.flatten()
        
        # Downsample by random sampling (much faster and uses less memory)
        n_samples = max(10000, int(sample_ratio * flat_img.size))
        indices = np.random.choice(flat_img.size, size=n_samples, replace=False)
        sample_data = flat_img[indices].reshape(-1, 1)
        
        # Initialize with K-means for faster convergence
        kmeans = KMeans(n_clusters=n_components, n_init=1, max_iter=100)
        kmeans.fit(sample_data)
        
        # Free up memory from kmeans
        kmeans_centers = kmeans.cluster_centers_.copy()
        del kmeans
        
        # Configure GMM with performance parameters
        gmm = GaussianMixture(
            n_components=n_components,
            random_state=42,
            n_init=1,
            max_iter=max_iter,
            tol=1e-3,
            means_init=kmeans_centers
        )
        
        # Fit on sample data
        gmm.fit(sample_data)
        
        # Extract and store only what we need from GMM
        means = gmm.means_.flatten().copy()
        bg_component = np.argmin(means)
        bg_mean = means[bg_component]
        bg_std = np.sqrt(gmm.covariances_[bg_component].flatten()[0])
        
        # Predict on full image for mask (this is memory intensive)
        pixel_labels = gmm.predict(flat_img.reshape(-1, 1))
        bg_mask = (pixel_labels == bg_component).reshape(channel_image.shape)
        
        # Free memory
        del sample_data, flat_img, pixel_labels
        
        # Store only what we need, not the full GMM model
        result = {
            'mean': bg_mean,
            'std': bg_std,
            'mask': bg_mask,
            'gmm': gmm,  # Store the fitted model for later use if needed
            'bg_percentage': np.sum(bg_mask) / bg_mask.size * 100,
            'component_means': means,
            'n_components': n_components,
            'bg_component': bg_component
        }
        
        # No need to store the full GMM model
        del gmm
        gc.collect()
        
        return result
        
    except Exception as e:
        print(f"Error in GMM background estimation: {str(e)}")
        traceback.print_exc()
        # Return fallback values
        return {'mean': 0, 'std': 0, 'mask': np.zeros_like(channel_image, dtype=bool)}

def estimate_background_gmm(channel_image, n_components=3, adaptive=False, max_components=6):
    """
    Estimate background using Gaussian Mixture Model
    
    Parameters:
    - channel_image: 2D array with image channel
    - n_components: Number of GMM components (default=3)
    - adaptive: If True, select optimal components using BIC (default=False)
    - max_components: Maximum number of components to try if adaptive=True
    
    Returns:
    - Dictionary with background statistics and fitted GMM model
    """
    
    # Flatten image for GMM
    flat_img = channel_image.flatten().reshape(-1, 1)
    
    # If adaptive component selection is requested
    if adaptive:
        bic_scores = []
        models = []
        
        # Try different numbers of components
        for n in range(1, max_components + 1):
            gmm = GaussianMixture(n_components=n, random_state=42, n_init=3)
            gmm.fit(flat_img)
            bic_scores.append(gmm.bic(flat_img))
            models.append(gmm)
        
        # Select model with lowest BIC score
        best_idx = np.argmin(bic_scores)
        gmm = models[best_idx]
        n_components = best_idx + 1  # Update n_components to the selected value
        print(f"Adaptive GMM selected {n_components} components with BIC: {bic_scores[best_idx]:.2f}")
    else:
        # Fit GMM model with specified components
        gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=3)
        gmm.fit(flat_img)
    
    # Identify background component (lowest mean)
    means = gmm.means_.flatten()
    bg_component = np.argmin(means)
    
    # Get background statistics
    bg_mean = means[bg_component]
    bg_std = np.sqrt(gmm.covariances_[bg_component].flatten()[0])
    
    # Create background mask 
    pixel_labels = gmm.predict(flat_img)
    bg_mask = (pixel_labels == bg_component).reshape(channel_image.shape)
    
    # Calculate background percentage
    bg_percentage = np.sum(bg_mask) / bg_mask.size * 100
    
    return {
        'mean': bg_mean,
        'std': bg_std,
        'mask': bg_mask,
        'gmm': gmm,  # Store the fitted model
        'bg_percentage': bg_percentage,
        'component_means': means,
        'n_components': n_components,
        'bg_component': bg_component
    }

def segment_cells_cellpose(image, config, bg_models=None):

    """Cell segmentation using CellPose with optimized memory usage"""
    io.logger_setup()
    
    # Extract the right channels for segmentation
    cyto_channel_idx = config.get('cytoplasm_channel', 4) - 1 
    nuc_channel_idx = config.get('nucleus_channel', 1) - 1
    
    # Choose model based on segmentation type
    if config.get('segmentation_type') == 'nuclei_only':
        model = models.Cellpose(gpu=config.get('use_gpu', True), model_type='nuclei')
        cellpose_channels = [0, 0]  # Standard for nuclei model
        
        # For single-channel segmentation
        img_to_segment = image[:,:,nuc_channel_idx].copy()
        
        # Apply background subtraction if needed
        if bg_models is not None and nuc_channel_idx in bg_models:
            ch_mean = bg_models[nuc_channel_idx]['mean']
            img_to_segment = np.clip(img_to_segment - ch_mean, 0, None)
            print(f"Channel {nuc_channel_idx}: Subtracted mean background {ch_mean:.2f}")
            
    else:
        model = models.Cellpose(gpu=config.get('use_gpu', True), model_type="cyto3")
        cellpose_channels = [2, 1]  # Standard for cyto model: 1=nuclei, 2=cyto
        
        # Create channels array with proper mapping between original and stacked indices
        channel_mapping = [cyto_channel_idx, nuc_channel_idx] 
        
        # Create the segmentation image 
        img_to_segment = np.stack([image[:,:,nuc_channel_idx], image[:,:,cyto_channel_idx]], axis=-1).copy()
        
        # Apply background subtraction with correct index mapping
        if bg_models is not None:
            for i, orig_idx in enumerate(channel_mapping):
                if orig_idx in bg_models:
                    ch_mean = bg_models[orig_idx]['mean']
                    img_to_segment[:,:,i] = np.clip(img_to_segment[:,:,i] - ch_mean, 0, None)
                    print(f"Channel {orig_idx} (stack position {i}): Subtracted mean background {ch_mean:.2f}")
    
    print(f"Running Cellpose with channels: {cellpose_channels}")
    print(f"Image shape for segmentation: {img_to_segment.shape}")
    
    # Run segmentation with debug info
    try:
        masks, flows, styles, diams = model.eval(
            img_to_segment, 
            channels=cellpose_channels,
            diameter=config.get('cell_diameter', 20.0),
            flow_threshold=config.get('flow_threshold', 0.4),
            cellprob_threshold=config.get('cellprob_threshold', 0.0),
            normalize=True,
            progress=True
        )
        
        # Free memory
        del img_to_segment
        if 'flows' in locals() and flows is not None:
            del flows
        if 'styles' in locals() and styles is not None:
            del styles
        if 'diams' in locals() and diams is not None:
            del diams
        gc.collect()
        
        print(f"Segmentation complete! Found {len(np.unique(masks))-1} objects")
        return masks
    
    except Exception as e:
        print(f"ERROR in Cellpose: {str(e)}")
        traceback.print_exc()
        return np.zeros(image.shape[:2], dtype=np.int32)

def segment_cells_with_downsampling(image, config, bg_models=None):
    """Segment cells with better memory management"""
    # Get downsampling factor from config
    downsample_factor = config.get('downsample_factor', 1.0)
    
    try:
        if downsample_factor >= 1.0:
            # Process at original resolution
            masks = segment_cells_cellpose(image, config, bg_models)
            return masks
        
        # Downsample image for processing
        small_image = resample_image(image, downsample_factor)
        
        # Adjust cell diameter for downsampled image
        small_config = config.copy()
        small_config['cell_diameter'] = config.get('cell_diameter', 20.0) * downsample_factor
        
        # Run segmentation on smaller image
        small_masks = segment_cells_cellpose(small_image, small_config, bg_models)
        
        # Free memory before upsampling
        del small_image
        gc.collect()
        
        # Upsample masks to original size
        masks_upscaled = resize(small_masks, image.shape[0:2], order=0, preserve_range=True)
        masks_upscaled = masks_upscaled.astype(np.int32)
        
        # Free memory
        del small_masks
        
        return masks_upscaled
        
    except Exception as e:
        print(f"Error in segmentation: {str(e)}")
        traceback.print_exc()
        # Return empty mask in case of error
        return np.zeros(image.shape[:2], dtype=np.int32)

def measure_cells_ctcf(image, cell_masks, bg_models, debug=False):
    """
    Measure CTCF for all cells and channels with progress tracking and debugging
    
    Parameters:
    - image: Multi-channel image array
    - cell_masks: Integer mask with cell labels
    - bg_models: Background models for each channel
    - debug: Enable detailed timing and debugging outputs
    """
    start_time = time.time()
    
    # Get properties of each cell
    props = measure.regionprops(cell_masks)
    measurements = []
    
    # Create progress bar
    total_cells = len(props)
    print(f"Measuring CTCF for {total_cells} cells across {image.shape[-1]} channels...")
    
    # Process cells with progress bar
    for i, prop in enumerate(tqdm(props, desc="Measuring cells", leave=False)):
        cell_start = time.time()
        
        # Basic cell properties
        cell_data = {
            'label': prop.label,
            'area': prop.area,
            'centroid': prop.centroid,
            'ctcf': {},
            'mean': {},
            'total': {},
            'bg_value': {}
        }
        
        # Get cell mask (this can be optimized by using prop.coords directly)
        mask = cell_masks == prop.label
        
        # Process all channels with a nested progress bar if debugging
        channels_iter = tqdm(range(image.shape[-1]), desc=f"Cell {i+1}/{total_cells} channels", 
                            leave=False, disable=not debug)
        
        for ch in channels_iter:
            # Extract channel and cell region
            channel = image[:,:,ch]
            
            # Use vectorized operation instead of creating intermediate mask
            # This is faster than doing channel[mask]
            cell_region = np.take(channel, prop.coords[:, 0], axis=0)
            cell_region = np.take(cell_region, prop.coords[:, 1], axis=1)
            
            # Background value
            bg_value = bg_models[ch]['mean']
            cell_data['bg_value'][ch] = bg_value
            
            # Calculate measurements
            total_intensity = np.sum(cell_region)
            mean_intensity = np.mean(cell_region)
            
            # CTCF = Integrated Density - (Area × Mean Background)
            ctcf = total_intensity - (prop.area * bg_value)
            
            # Store results
            cell_data['total'][ch] = total_intensity
            cell_data['mean'][ch] = mean_intensity
            cell_data['ctcf'][ch] = ctcf
            
        measurements.append(cell_data)
        
        # Print debug info periodically
        if debug and (i == 0 or i == total_cells-1 or i % max(1, total_cells//100) == 0):
            cell_time = time.time() - cell_start
            print(f"  Cell {i+1}/{total_cells} processed in {cell_time:.3f}s")
    
    # Final timing
    total_time = time.time() - start_time
    cells_per_sec = total_cells / total_time
    print(f"CTCF measurement complete: {total_cells} cells in {total_time:.2f}s ({cells_per_sec:.2f} cells/sec)")
    
    return measurements

def measure_cells_ctcf_gpu(image, cell_masks, bg_models, debug=False):
    """GPU-accelerated CTCF measurement for improved performance"""
    
    start_time = time.time()
    
    # Convert arrays to PyTorch tensors on GPU
    device = torch.device('cuda')
    image_tensor = torch.from_numpy(image).to(device)
    mask_tensor = torch.from_numpy(cell_masks).to(device)
    
    # Get unique cell IDs for processing
    cell_ids = torch.unique(mask_tensor)[1:]  # Skip 0 (background)
    total_cells = len(cell_ids)
    print(f"Measuring CTCF for {total_cells} cells across {image.shape[-1]} channels on GPU...")
    
    # Prepare results container
    measurements = []
    
    # Process in batches if there are many cells
    batch_size = 500  # Adjust based on your GPU memory
    for batch_start in tqdm(range(0, total_cells, batch_size), desc="Processing cell batches"):
        batch_end = min(batch_start + batch_size, total_cells)
        batch_ids = cell_ids[batch_start:batch_end]
        
        # Process each cell in the batch (can parallelize this loop too)
        for cell_idx in range(len(batch_ids)):
            cell_id = batch_ids[cell_idx].item()
            
            # Create binary mask for this cell
            cell_mask = (mask_tensor == cell_id)
            cell_area = torch.sum(cell_mask).item()
            
            # Calculate centroid (still using CPU as this is not the bottleneck)
            # You could implement a GPU version if needed
            y_indices, x_indices = torch.where(cell_mask)
            centroid_y = torch.mean(y_indices.float()).item()
            centroid_x = torch.mean(x_indices.float()).item()
            
            cell_data = {
                'label': cell_id,
                'area': cell_area,
                'centroid': (centroid_y, centroid_x),
                'ctcf': {},
                'mean': {},
                'total': {},
                'bg_value': {}
            }
            
            # Process all channels using GPU operations
            for ch in range(image_tensor.shape[2]):
                channel_data = image_tensor[:, :, ch]
                bg_value = bg_models[ch]['mean']
                cell_data['bg_value'][ch] = bg_value
                
                # GPU-accelerated measurements
                cell_pixels = channel_data[cell_mask]
                total_intensity = torch.sum(cell_pixels).item()
                mean_intensity = torch.mean(cell_pixels).item()
                ctcf = total_intensity - (cell_area * bg_value)
                
                cell_data['total'][ch] = total_intensity
                cell_data['mean'][ch] = mean_intensity
                cell_data['ctcf'][ch] = ctcf
            
            measurements.append(cell_data)
        
        # Free GPU memory after each batch
        torch.cuda.empty_cache()
    
    # Final timing
    total_time = time.time() - start_time
    cells_per_sec = total_cells / total_time
    print(f"GPU CTCF measurement complete: {total_cells} cells in {total_time:.2f}s ({cells_per_sec:.2f} cells/sec)")
    
    return measurements


### Crop Section testing 
In this section the functions for testing on a small cropped picture of the image are dealigned

In [5]:
def crop_central_region(image, crop_ratio=0.1):
    """
    Crop the central region of an image for quick segmentation testing
    
    Parameters:
    - image: The input image (H, W, C)
    - crop_ratio: Size of the crop relative to original image (0.1 = 10%)
    
    Returns:
    - cropped_image: The central cropped region
    - crop_coords: (y_start, y_end, x_start, x_end) for reference
    """
    # Get image dimensions
    h, w, c = image.shape
    
    # Calculate crop size
    crop_h = int(h * crop_ratio)
    crop_w = int(w * crop_ratio)
    
    # Calculate central coordinates
    center_y, center_x = h // 2, w // 2
    
    # Calculate crop boundaries
    y_start = center_y - (crop_h // 2)
    y_end = center_y + (crop_h // 2)
    x_start = center_x - (crop_w // 2)
    x_end = center_x + (crop_w // 2)
    
    # Ensure coordinates are within image bounds
    y_start = max(0, y_start)
    y_end = min(h, y_end)
    x_start = max(0, x_start)
    x_end = min(w, x_end)
    
    # Extract crop
    cropped_image = image[y_start:y_end, x_start:x_end, :]
    
    return cropped_image, (y_start, y_end, x_start, x_end)    

def test_segmentation_on_crop(image_path, output_dir, config, crop_ratio=0.1):
        """
        Test segmentation on a central crop of an image
        
        Parameters:
        - image_path: Path to the input image
        - output_dir: Directory to save results
        - config: Configuration dictionary
        - crop_ratio: Size of the crop relative to original image (0.1 = 10%)
        
        Returns:
        - Dictionary with segmentation results and parameters
        """
        # Load image and normalize channels
        image = tiff.imread(image_path)
        img_name = os.path.basename(image_path)
        img_base = os.path.splitext(img_name)[0]
        
        print(f"\nTesting segmentation on cropped region of: {img_name}")
        
        # Move the shortest axis (channels) to the last index if needed
        shortest_axis = np.argmin(image.shape)
        image = np.moveaxis(image, shortest_axis, -1)
        
        # Crop central region
        cropped_image, crop_coords = crop_central_region(image, crop_ratio)
        y_start, y_end, x_start, x_end = crop_coords
        
        print(f"Original image shape: {image.shape}")
        print(f"Cropped region shape: {cropped_image.shape}")
        print(f"Crop coordinates: (y={y_start}:{y_end}, x={x_start}:{x_end})")
        
        # Create output directory for this test if it doesn't exist
        crop_output_dir = os.path.join(output_dir, f"{img_base}_crop_test")
        os.makedirs(crop_output_dir, exist_ok=True)
        
        # 1. Estimate background for the cropped region
        print("Estimating background using GMM...")
        bg_models = {}
        for ch in range(cropped_image.shape[-1]):
            channel_data = cropped_image[:,:,ch].copy()
            bg_models[ch] = fast_estimate_background_gmm(channel_data)
            
            # Save background mask visualization
            visualize_background_mask(channel_data, bg_models[ch], 
                                     os.path.join(crop_output_dir, f"crop_bg_mask_ch{ch+1}.png"))
        
        # 2. Segment cells on the cropped region
        cell_masks = segment_cells_with_downsampling(cropped_image, config, bg_models)
        
        # 3. Measure CTCF for each cell in the cropped region
        cell_measurements = measure_cells_ctcf(cropped_image, cell_masks, bg_models)
        
        # Save visualization of the segmentation results
        create_visualization(cropped_image, cell_masks, cell_measurements, 
                            os.path.join(crop_output_dir, f"{img_base}_crop_segmentation.png"), 
                            debug=True)
        
        # Create a comparison visualization showing where the crop is from
        plt.figure(figsize=(12, 6))
        
        # Show original with crop region highlighted
        plt.subplot(1, 2, 1)
        # Use first channel for display or create a composite
        if image.shape[-1] >= 3:
            display_img = np.zeros((image.shape[0], image.shape[1], 3))
            for i in range(min(3, image.shape[-1])):
                ch_data = exposure.equalize_adapthist(image[:,:,i])
                display_img[:,:,i] = ch_data
        else:
            display_img = exposure.equalize_adapthist(image[:,:,0])
        
        plt.imshow(display_img)
        plt.gca().add_patch(plt.Rectangle((x_start, y_start), 
                                         x_end - x_start, 
                                         y_end - y_start, 
                                         fill=False, 
                                         edgecolor='red', 
                                         linewidth=2))
        plt.title('Original Image with Crop Region')
        
        # Show the cropped region with segmentation overlay
        plt.subplot(1, 2, 2)
        # Create overlay of segmentation on image
        if cropped_image.shape[-1] >= 3:
            crop_display = np.zeros((cropped_image.shape[0], cropped_image.shape[1], 3))
            for i in range(min(3, cropped_image.shape[-1])):
                ch_data = exposure.equalize_adapthist(cropped_image[:,:,i])
                crop_display[:,:,i] = ch_data
        else:
            crop_display = exposure.equalize_adapthist(cropped_image[:,:,0])
        
        plt.imshow(crop_display)
        # Add cell mask overlay
        plt.imshow(cell_masks > 0, alpha=0.7, cmap='cool')
        plt.title(f'Segmentation on Cropped Region ({len(cell_measurements)} cells)')
        
        plt.tight_layout()
        plt.savefig(os.path.join(crop_output_dir, f"{img_base}_crop_location.png"), dpi=150)
        plt.close()
        
        # Save cell measurements to CSV
        cell_df = pd.DataFrame([
            {
                'cell_id': cell['label'],
                'area': cell['area'],
                **{f'channel_{ch+1}_ctcf': cell['ctcf'][ch] for ch in range(cropped_image.shape[-1])},
                **{f'channel_{ch+1}_mean': cell['mean'][ch] for ch in range(cropped_image.shape[-1])},
                'centroid_x': cell['centroid'][1],
                'centroid_y': cell['centroid'][0]
            }
            for cell in cell_measurements
        ])
        cell_df.to_csv(os.path.join(crop_output_dir, f"{img_base}_crop_cells.csv"), index=False)
        
        # Return info about the test
        return {
            'image_name': img_name,
            'crop_region': crop_coords,
            'cell_count': len(cell_measurements),
            'output_dir': crop_output_dir
        }

# Test segmentation on a cropped region before full processing
def test_segmentation_parameters(image_path, config, crop_ratio=0.1):
    """Test segmentation parameters on a cropped region of an image"""
    # Create a temporary output directory
    output_dir = os.path.join(os.path.dirname(image_path), "segmentation_tests")
    os.makedirs(output_dir, exist_ok=True)
    
    # Run the crop test
    test_result = test_segmentation_on_crop(
        image_path=image_path,
        output_dir=output_dir,
        config=config,
        crop_ratio=crop_ratio
    )
    
    print(f"\n✓ Segmentation test complete!")
    print(f"  - Found {test_result['cell_count']} cells in the cropped region")
    print(f"  - Results saved to: {test_result['output_dir']}")
    print("\nTIP: Review the results and adjust segmentation parameters in config as needed")
    
    return test_result

## Process Images
In this section firstly the configuratin will be set for the segmentation, and a test can be done for an individual file. On the second part, a batch processing of multiple files can be done.

In [12]:
# Define the configuration for the pipeline
config = {
    # CellPose settings
    'segmentation_type': 'cyto_and_nuclei',  # or 'nuclei_only'
    'cytoplasm_channel': 4,  # Far Red channel for cytoplasm/membrane
    'nucleus_channel': 1,    # Blue channel (DAPI) for nuclei
    'cell_diameter': 30.0,     # Approximate diameter in pixels
    'use_gpu': True,         # Use GPU acceleration
    'flow_threshold': 0.4,   # Flow threshold for CellPose
    'cellprob_threshold': 0.6,  # Cell probability threshold for CellPose
    'downsample_factor': 0.5,  # Downsample factor for speed (1.0 = no downsampling)
    
    # Visualization settings
    'visualize_steps': True,  # Set to False if you don't want intermediate visualizations
    
    # Other pipeline settings
    'channels_of_interest': [0, 1, 2, 3]  # All channels to measure
}

### Cropped segmentation and background substraction testing

In [15]:
folder_path_test = select_folder()
print(f'>>> Selected folder: {folder_path_test}')

>>> Selected folder: H:/01_Colaboration/02_Slide_Scanner/20250605_TestFolder/Exp1


In [16]:
# Example usage in your script
image_paths = glob.glob(os.path.join(folder_path_test, "*.tiff"))
if len(image_paths) > 0:
    # Test segmentation on first image before batch processing
    test_result = test_segmentation_parameters(image_paths[0], config, crop_ratio=0.1)
    
else:
    print(f"No TIFF images found in {folder_path_test}")


Testing segmentation on cropped region of: EXP10_2_1_1_1_2_3_scFl_DoxCont_Light-OME_s5.ome.tiff
Original image shape: (11232, 13066, 4)
Cropped region shape: (1122, 1306, 4)
Crop coordinates: (y=5055:6177, x=5880:7186)
Estimating background using GMM...
creating new log file
2025-04-09 21:14:25,498 [INFO] WRITING LOG OUTPUT TO C:\Users\s.molina\.cellpose\run.log
2025-04-09 21:14:25,499 [INFO] 
cellpose version: 	3.1.1.1 
platform:       	win32 
python version: 	3.12.3 
torch version:  	2.8.0.dev20250405+cu128
2025-04-09 21:14:25,502 [INFO] ** TORCH CUDA version installed and working. **
2025-04-09 21:14:25,503 [INFO] >>>> using GPU (CUDA)
2025-04-09 21:14:25,504 [INFO] >> cyto3 << model set to be used
2025-04-09 21:14:25,601 [INFO] >>>> loading model C:\Users\s.molina\.cellpose\models\cyto3
2025-04-09 21:14:25,764 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
Channel 3 (stack position 0): Subtracted mean background 383.36
Channel 0 (stack position 

CTCF measurement complete: 49 cells in 2.44s (20.07 cells/sec)
Starting visualization for H:/01_Colaboration/02_Slide_Scanner/20250605_TestFolder/Exp1\segmentation_tests\EXP10_2_1_1_1_2_3_scFl_DoxCont_Light-OME_s5.ome_crop_test\EXP10_2_1_1_1_2_3_scFl_DoxCont_Light-OME_s5.ome_crop_segmentation.png...
Created figure with size 20x4 inches at 300 DPI
Processing 4 channels and 49 cells
Rendering segmentation mask...
Adding cell labels...


Processing channels:  25%|██▌       | 1/4 [00:00<00:00,  3.64it/s]

  Channel 1 rendered in 0.28s


Processing channels:  50%|█████     | 2/4 [00:00<00:00,  3.64it/s]

...

Processing channels:  75%|███████▌  | 3/4 [00:00<00:00,  3.70it/s]

  Channel 3 rendered in 0.26s


Processing channels: 100%|██████████| 4/4 [00:01<00:00,  3.72it/s]

  Channel 4 rendered in 0.26s
Saving figure...


Visualization saved in 4.91s
Total visualization time: 6.47s

✓ Segmentation test complete!
  - Found 49 cells in the cropped region
  - Results saved to: H:/01_Colaboration/02_Slide_Scanner/20250605_TestFolder/Exp1\segmentation_tests\EXP10_2_1_1_1_2_3_scFl_DoxCont_Light-OME_s5.ome_crop_test

TIP: Review the results and adjust segmentation parameters in config as needed


### Batch Processing 

In [ ]:
batch_folder = select_folder()
print(f'>>> Selected folder: {batch_folder}')

In [ ]:
# Process all experiments in the selected folder
batch_results = process_experiments_batch(batch_folder, config)
print(f'>>> Batch processing complete. Results saved to: {batch_results}')